In [1]:
import pandas as pd
import json

### Link to the dataset

https://www.kaggle.com/datasets/wordsforthewise/lending-club/data?select=accepted_2007_to_2018Q4.csv.gz

In [2]:
#rejected_loan_data = pd.read_csv('data/rejected_2007_to_2018Q4.csv.gz')
loan_data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz')

C:\Users\abarg\AppData\Local\Temp\ipykernel_14800\37491141.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz')


In [3]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


`issue_d` Month and year when the loan was issued

`addr_state` State provided by the borrower in the loan application

`loan_status` Current status of the loan

In [4]:
loan_data['loan_status'].value_counts()

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

In [5]:
# # Check null values
# loan_data[loan_data['loan_status'].isnull()]

In [6]:
# loan_data[loan_data['loan_status'].isnull()].index == loan_data[loan_data['issue_d'].isnull()].index

In [7]:
loan_data['issue_d'].head()

0    Dec-2015
1    Dec-2015
2    Dec-2015
3    Dec-2015
4    Dec-2015
Name: issue_d, dtype: object

In [8]:
# Drop unnecessary id columns
loan_data = loan_data.drop(columns=['id', 'member_id'])

# Drop null values in issue_d (and target) column
loan_data = loan_data[loan_data['issue_d'].notnull()]

In [9]:
# Add year column from issue date
loan_data['issue_year'] = loan_data['issue_d'].str[-4:].astype(int)
#loan_data['issue_year'] = loan_data['issue_d'].astype('datetime64[ns]').dt.year

In [10]:
loan_data[['issue_d', 'issue_year']].sample(10)

,issue_d,issue_year
1249270,Jun-2014,2014
1715171,Feb-2017,2017
1822744,Jul-2013,2013
573764,Aug-2017,2017
671407,Jun-2016,2016
2165243,Dec-2016,2016
1198785,Sep-2014,2014
813266,Aug-2018,2018
1038079,Mar-2016,2016
1233618,Jul-2014,2014


### Import yearly unemployment rate data

### Link to the datasets

https://fred.stlouisfed.org/release/tables?rid=116&eid=840687

In [11]:
# Load unemployment rate data from multiple csv files and merge them on observation_date
paths = ["data/unemployment_rate_0.csv", "data/unemployment_rate_1.csv", "data/unemployment_rate_2.csv", "data/unemployment_rate_3.csv", "data/unemployment_rate_4.csv"]

y_unemployment_df = pd.read_csv(paths[0])
for path in paths[1:]:
    df = pd.read_csv(path)
    y_unemployment_df = y_unemployment_df.merge(df, on="observation_date", how="outer")

In [12]:
# Add year column from observation date
y_unemployment_df['year'] = y_unemployment_df['observation_date'].astype('datetime64[ns]').dt.year

# Drop observation date column
y_unemployment_df = y_unemployment_df.drop(columns=['observation_date'])

# Import column names
with open('data/unemployment_rate_dict.json') as f:
    states_ref = json.load(f)

states_names = list(states_ref.keys())
col_names = states_names + ["year"]

# Rename columns
y_unemployment_df.columns = col_names

# Reshape unemployment data to long format
y_unemployment_df = y_unemployment_df.melt(id_vars=["year"], var_name="state", value_name="y_unemployment_rate")

In [13]:
y_unemployment_df.head()

,year,state,y_unemployment_rate
0,1976,AL,6.7
1,1977,AL,7.1
2,1978,AL,6.5
3,1979,AL,7.2
4,1980,AL,8.7


### Merge unemployment rates with loan data

In [14]:
data = loan_data.merge(
    y_unemployment_df,
    left_on=["issue_year", "addr_state"],
    right_on=["year", "state"],
    how="left"
).drop(columns=["year", "state"])

In [15]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,issue_year,y_unemployment_rate
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,5.4
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,3.0
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,6.0
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,5.7
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,5.4
